# Теперь с нормализацией и работой с данными + торч

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [ ]:
import catboost

In [ ]:
train=pd.read_csv("./train_dataset_train (1).csv")

In [ ]:
test=pd.read_csv("./test_dataset_test.csv")

In [ ]:
df=train.copy()

In [ ]:
tes=test.copy()

In [ ]:
cat=['Пол','Основания','Изучаемый_Язык','Дата_Рождения','Уч_Заведение','Где_Находится_УЗ','Страна_Родители','Страна_ПП','Регион_ПП','Город_ПП']

In [ ]:
num=['Код_группы','Год_Поступления','Год_Окончания_УЗ','Общежитие','Наличие_Матери','Наличие_Отца','Село','Иностранец','КодФакультета','СрБаллАттестата']

# Информация о данных

In [ ]:
df.describe(include="all")

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Общежитие,Наличие_Матери,Наличие_Отца,Страна_Родители,Опекунство,Село,Иностранец,КодФакультета,СрБаллАттестата,Статус
count,13584.000000,13584.000000,13584.000000,13577,13584,12758,13584,12351,12203,12284.000000,...,13498.000000,13584.000000,13584.000000,13126,13584.000000,13498.000000,13274.000000,13584.000000,13584.000000,13584.000000
unique,NaN,NaN,NaN,3,5,7,5044,3646,2110,NaN,...,NaN,NaN,NaN,19,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,Жен,ОО,Английский язык,1995-08-07 00:00:00.000,ФГБОУ ВО Алтайский государственный университет,"Алтайский край, Барнаул г",NaN,...,NaN,NaN,NaN,Россия,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,8133,5084,10450,16,1359,2413,NaN,...,NaN,NaN,NaN,12239,NaN,NaN,NaN,NaN,NaN,NaN
mean,70096.056905,18302.499926,2014.992123,NaN,NaN,NaN,NaN,NaN,NaN,2013.876018,...,0.219884,0.511484,0.509570,NaN,0.000442,0.229441,0.070589,32.500368,73.223274,3.426458
std,12705.894531,2540.417956,2.595339,NaN,NaN,NaN,NaN,NaN,NaN,4.247111,...,0.414184,0.499886,0.499927,NaN,0.021013,0.420489,0.256147,8.458150,258.045027,1.071765
min,44632.000000,11550.000000,2001.000000,NaN,NaN,NaN,NaN,NaN,NaN,1973.000000,...,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,24.000000,0.000000,-1.000000
25%,63496.750000,16901.000000,2013.000000,NaN,NaN,NaN,NaN,NaN,NaN,2013.000000,...,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,26.000000,47.000000,3.000000
50%,71401.500000,18534.500000,2015.000000,NaN,NaN,NaN,NaN,NaN,NaN,2015.000000,...,0.000000,1.000000,1.000000,NaN,0.000000,0.000000,0.000000,28.000000,61.000000,4.000000
75%,78404.250000,20599.000000,2016.000000,NaN,NaN,NaN,NaN,NaN,NaN,2016.000000,...,0.000000,1.000000,1.000000,NaN,0.000000,0.000000,0.000000,40.000000,75.000000,4.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13584 entries, 0 to 13583
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                13584 non-null  int64  
 1   Код_группы        13584 non-null  int64  
 2   Год_Поступления   13584 non-null  int64  
 3   Пол               13577 non-null  object 
 4   Основания         13584 non-null  object 
 5   Изучаемый_Язык    12758 non-null  object 
 6   Дата_Рождения     13584 non-null  object 
 7   Уч_Заведение      12351 non-null  object 
 8   Где_Находится_УЗ  12203 non-null  object 
 9   Год_Окончания_УЗ  12284 non-null  float64
 10  Пособие           12576 non-null  float64
 11  Страна_ПП         13237 non-null  object 
 12  Регион_ПП         12966 non-null  object 
 13  Город_ПП          13141 non-null  object 
 14  Общежитие         13498 non-null  float64
 15  Наличие_Матери    13584 non-null  int64  
 16  Наличие_Отца      13584 non-null  float6

# Work with data

Уберём лишние столбцы

In [ ]:
df=df.drop(["Пособие","Опекунство"],axis=1)

In [ ]:
df=df.drop("ID",axis=1)

Приведём балл к одной шкале

In [ ]:
df["СрБаллАттестата"]=df["СрБаллАттестата"].apply(lambda x: 2*x if 0<x<=5 else x/10 if 10<x<=100 else x/100 if 100<x<=1000 else x/1000)

Дропнем Nan-ы

In [ ]:
df[cat]=df[cat].fillna("Nan")
df[num]=df[num].fillna(0)

Разберёмся с г в городах

In [ ]:
df["Город_ПП"].value_counts()

Барнаул г         3735
г. Барнаул        1727
Барнаул           1174
Nan                443
Бийск г            246
                  ... 
Марушка              1
с. Гилево            1
Чернышевский п       1
Кисилёвск г          1
Шила с               1
Name: Город_ПП, Length: 1766, dtype: int64

In [ ]:
df.loc[:,"Город_ПП"]=df.loc[:,"Город_ПП"].apply(lambda x: max(x.split(),key=len,default="Nan"))

In [ ]:
df["Город_ПП"].value_counts()

Барнаул        6791
Бийск           537
Nan             485
Новоалтайск     396
Рубцовск        190
               ... 
Бурганды          1
Хохряки           1
-                 1
Логовской         1
Шила              1
Name: Город_ПП, Length: 1196, dtype: int64

Разберёмся с языками

In [ ]:
df.loc[:,"Изучаемый_Язык"]=df.loc[:,"Изучаемый_Язык"].apply(lambda x: x.split()[0][:3] if (len(x.split())==1 or len(x.split())==2) else x.split()[-1][1:4])

уберем родителей

In [ ]:
df=df.drop("Страна_Родители",axis=1)

In [ ]:
def reg(x):
  if "Республика" in x:
    x.remove("Республика")
  if "республика" in x:
    x.remove("республика")
  if "Провинция" in x:
    x.remove("Провинция")
  if "провинция" in x:
    x.remove("провинция")
  return x

In [ ]:
df.loc[:,"Регион_ПП"]=df.loc[:,"Регион_ПП"].apply(lambda x: max(reg(x.split()),key=len))

In [ ]:
df.head()

,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,Страна_ПП,Регион_ПП,Город_ПП,Общежитие,Наличие_Матери,Наличие_Отца,Село,Иностранец,КодФакультета,СрБаллАттестата,Статус
0,16019,2015,Жен,ОО,Анг,1998-02-08 00:00:00.000,"МБОУ ""СОШ №59""","Алтайский край, Барнаул г",2014.0,Россия,Алтайский,Барнаул,0.0,0,0.0,0.0,0.0,41.0,8.588,3
1,14895,2010,Муж,ЦН,Nan,1992-04-18 00:00:00.000,Nan,Nan,0.0,Россия,Алтайский,Барнаул,0.0,1,1.0,0.0,0.0,28.0,6.700,4
2,20103,2017,Жен,ДН,Анг,1999-03-21 00:00:00.000,МБОУ Алтайская СОШ №5,"Алтайский край, Алтайский р-н, Алтайское с",2017.0,Россия,Алтайский,Алтайское,0.0,1,1.0,0.0,0.0,41.0,8.266,4
3,19838,2017,Жен,БН,Анг,1994-12-08 00:00:00.000,ФГБОУ ВО Алтайский государственный университет,"Алтайский край, г. Барнаул",2017.0,Россия,Алтайский,Барнаул,0.0,1,1.0,0.0,0.0,35.0,7.400,4
4,19803,2017,Жен,БН,Анг,1995-06-28 00:00:00.000,ФГБОУ ВО Алтайский государственный университет,"Алтайский край, г. Барнаул",2017.0,Россия,Алтайский,Барнаул,0.0,1,1.0,0.0,0.0,28.0,5.300,4


# способ один. через торч без уч заведения

In [ ]:
#df=df.drop("Уч_Заведение",axis=1)

In [ ]:
#df.head()

# Уч заведения в вузы и школы

In [ ]:
#gfsd

# Даммис

In [ ]:
#df=pd.get_dummies(df)

# пишем нейронку

In [ ]:
data2=df.copy()

In [ ]:
y=df["Статус"]

In [ ]:
y=y.apply(lambda x: 0if x<0 else 1 if x==3 else 2 if x==4 else x)

In [ ]:
df=df.drop(["Статус"],axis=1)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(df, y, train_size=0.8, random_state=1, stratify=y)

In [ ]:
X_train = torch.FloatTensor(X_train.values)
X_valid = torch.FloatTensor(X_valid.values)
y_train = torch.LongTensor(y_train.values)
y_valid = torch.LongTensor(y_valid.values)

In [ ]:
y

In [ ]:
y.value_counts()

In [ ]:
class eduNet(torch.nn.Module):
    def __init__(self, n_hidden_neurons):
        super(eduNet, self).__init__()

        self.fc1 = torch.nn.Linear(7785, n_hidden_neurons)
        self.activ1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(n_hidden_neurons, n_hidden_neurons)
        self.activ2 = torch.nn.Sigmoid()
        self.fc3 = torch.nn.Linear(n_hidden_neurons, 3)
        self.sm = torch.nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.activ1(x)
        x = self.fc2(x)
        x = self.activ2(x)
        x = self.fc3(x)
        return x

    def inference(self, x):
        x = self.forward(x)
        x = self.sm(x)
        return x

edu = eduNet(100)

In [ ]:
!pip install torchmetrics

In [ ]:
import torch
from torchmetrics import F1Score

In [ ]:
loss=F1Score(num_classes=3, average="macro")

optimizer = torch.optim.SGD(edu.parameters(),
                             lr=1.0e-2,momentum=0.7)

In [ ]:
batch_size = 10

for epoch in range(5000):
    order = np.random.permutation(len(X_train))
    for start_index in range(0, len(X_train), batch_size):
        optimizer.zero_grad()

        batch_indexes = order[start_index:start_index+batch_size]

        x_batch = X_train[batch_indexes]
        y_batch = y_train[batch_indexes]

        preds = edu.forward(x_batch)

        loss_value = loss(preds, y_batch)
        loss_value.backward()

        optimizer.step()

    if epoch % 100 == 0:
        test_preds = edu.forward(X_valid)
        test_preds = test_preds.argmax(dim=1)
        print((test_preds == y_valid).float().mean())

# А теперь catboost

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13584 entries, 0 to 13583
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Код_группы        13584 non-null  int64  
 1   Год_Поступления   13584 non-null  int64  
 2   Пол               13584 non-null  object 
 3   Основания         13584 non-null  object 
 4   Изучаемый_Язык    13584 non-null  object 
 5   Дата_Рождения     13584 non-null  object 
 6   Уч_Заведение      13584 non-null  object 
 7   Где_Находится_УЗ  13584 non-null  object 
 8   Год_Окончания_УЗ  13584 non-null  float64
 9   Страна_ПП         13584 non-null  object 
 10  Регион_ПП         13584 non-null  object 
 11  Город_ПП          13584 non-null  object 
 12  Общежитие         13584 non-null  float64
 13  Наличие_Матери    13584 non-null  int64  
 14  Наличие_Отца      13584 non-null  float64
 15  Село              13584 non-null  float64
 16  Иностранец        13584 non-null  float6

In [ ]:
cat=["Пол",'Основания','Изучаемый_Язык','Дата_Рождения','Уч_Заведение','Где_Находится_УЗ','Страна_ПП','Регион_ПП','Город_ПП']

In [ ]:
df=data2.copy()
y=df["Статус"]
df=df.drop(["Статус"],axis=1)
X_train, X_valid, y_train, y_valid = train_test_split(df, y, train_size=0.8, random_state=1, stratify=y)
boosting_model = catboost.CatBoostClassifier(n_estimators=10000, learning_rate=0.011, loss_function="MultiClassOneVsAll",
                                             cat_features=cat,task_type="GPU")

boosting_model.fit(X_train,y_train,plot=True,verbose=35,eval_set=(X_valid,y_valid))
y_train_predicted = boosting_model.predict_proba(X_train)[:, 1]
y_valid_predicted = boosting_model.predict_proba(X_valid)[:, 1]
print(f"Model params: {boosting_model.get_params()}")

In [ ]:
pred=boosting_model.predict(X_valid)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_valid,pred,average="macro",zero_division=0)

0.7323060803162118

In [ ]:
params = {'learning_rate':np.arange(0.01,0.03,0.01), 'iterations':np.arange(700,701,1), 'depth':np.arange(7,8,1),'min_data_in_leaf': np.arange(1,2,2)}

In [ ]:
boosting_model2 = catboost.CatBoostClassifier(cat_features=cat, random_seed=1, verbose=False,task_type="GPU",loss_function="MultiClassOneVsAll")
result = boosting_model2.grid_search(params, X_train, y_train)
y_train_predicted2 = boosting_model2.predict_proba(X_train)[:, 1]
y_valid_predicted2 = boosting_model2.predict_proba(X_valid)[:, 1]

bestTest = 0.2545900459
bestIteration = 699
0:	loss: 0.2545900	best: 0.2545900 (0)	total: 9.91s	remaining: 9.91s
bestTest = 0.2432451432
bestIteration = 698
1:	loss: 0.2432451	best: 0.2432451 (1)	total: 17.5s	remaining: 0us
Estimating final quality...
Training on fold [0/3]
bestTest = 0.2408982481
bestIteration = 699
Training on fold [1/3]
bestTest = 0.2391159306
bestIteration = 699
Training on fold [2/3]
bestTest = 0.2379000373
bestIteration = 699


In [ ]:
result["params"]

{'min_data_in_leaf': 1, 'depth': 7, 'iterations': 700, 'learning_rate': 0.02}

In [ ]:
pred=boosting_model2.predict(X_valid)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_valid,pred,average="macro",zero_division=0)

0.7346358373649954

# Работаем над тестом

In [ ]:
df=pd.read_csv("./test_dataset_test.csv")

NameError: ignored

In [ ]:
df=df.drop(["Пособие","Опекунство"],axis=1)

In [ ]:
df=df.drop("ID",axis=1)

Приведём балл к одной шкале

In [ ]:
df["СрБаллАттестата"]=df["СрБаллАттестата"].apply(lambda x: 2*x if 0<x<=5 else x/10 if 10<x<=100 else x/100 if 100<x<=1000 else x/1000)

Дропнем Nan-ы

In [ ]:
df[cat]=df[cat].fillna("Nan")
df[num]=df[num].fillna(0)

Разберёмся с г в городах

In [ ]:
df["Город_ПП"].value_counts()

In [ ]:
df.loc[:,"Город_ПП"]=df.loc[:,"Город_ПП"].apply(lambda x: max(x.split(),key=len,default="Nan"))

In [ ]:
df["Город_ПП"].value_counts()

Разберёмся с языками

In [ ]:
df.loc[:,"Изучаемый_Язык"]=df.loc[:,"Изучаемый_Язык"].apply(lambda x: x.split()[0][:3] if (len(x.split())==1 or len(x.split())==2) else x.split()[-1][1:4])

уберем родителей

In [ ]:
df=df.drop("Страна_Родители",axis=1)

In [ ]:
def reg(x):
  if "Республика" in x:
    x.remove("Республика")
  if "республика" in x:
    x.remove("республика")
  if "Провинция" in x:
    x.remove("Провинция")
  if "провинция" in x:
    x.remove("провинция")
  return x

In [ ]:
df.loc[:,"Регион_ПП"]=df.loc[:,"Регион_ПП"].apply(lambda x: max(reg(x.split()),key=len))

# Импортим

In [ ]:
df_submission=pd.read_csv("./sample_submission.csv")

In [ ]:
df_test[num]=df_test[num].fillna(0)

In [ ]:
df_test[cat]=df_test[cat].fillna("Nan")

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6691 entries, 0 to 6690
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                6691 non-null   int64  
 1   Код_группы        6691 non-null   int64  
 2   Год_Поступления   6691 non-null   int64  
 3   Пол               6691 non-null   object 
 4   Основания         6691 non-null   object 
 5   Изучаемый_Язык    6691 non-null   object 
 6   Дата_Рождения     6691 non-null   object 
 7   Уч_Заведение      6691 non-null   object 
 8   Где_Находится_УЗ  6691 non-null   object 
 9   Год_Окончания_УЗ  6691 non-null   float64
 10  Пособие           6691 non-null   float64
 11  Страна_ПП         6691 non-null   object 
 12  Регион_ПП         6691 non-null   object 
 13  Город_ПП          6691 non-null   object 
 14  Общежитие         6691 non-null   float64
 15  Наличие_Матери    6691 non-null   int64  
 16  Наличие_Отца      6691 non-null   float64


In [ ]:
df_test_pred=boosting_model3.predict(df_test[features])

In [ ]:
df_submission["Статус"]=df_test_pred
df_submission.head()

,ID,Статус
0,64996,4
1,71837,4
2,86587,3
3,73673,3
4,54709,4


In [ ]:
df_submission.to_csv("./sample_submission.csv",index=False)